In [40]:
import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedKFold
from joblib import load, dump
from pathlib2 import Path
from pdb import set_trace

In [59]:
df = pd.read_csv('train_labels_as_strings.csv')
df_pi = pd.read_csv('DATA_with_PATIENT_ID.csv')[['fn', 'PatientID']]
df_new = pd.merge(df, df_pi, on='fn')
df_new_lb = df_new[['PatientID', 'labels']].copy()

In [42]:
df_new_lb

,PatientID,labels
0,ID_a449357f,NaN
1,ID_363d5865,NaN
2,ID_9c2b4bd7,NaN
3,ID_3ae81c2d,NaN
4,ID_c1867feb,NaN
...,...,...
674252,ID_cd9b1eb3,NaN
674253,ID_39702b69,NaN
674254,ID_c9eb1b4d,NaN
674255,ID_141ef473,NaN


In [43]:
df_new_lb['labels'].value_counts()

subdural any                                                            28909
subarachnoid any                                                        14717
intraparenchymal any                                                    13932
intraventricular any                                                     8914
intraparenchymal intraventricular any                                    6330
subarachnoid subdural any                                                4654
intraparenchymal subarachnoid any                                        3611
intraventricular subarachnoid any                                        3226
intraparenchymal subdural any                                            2952
intraparenchymal subarachnoid subdural any                               2017
intraparenchymal intraventricular subarachnoid any                       1761
epidural any                                                             1497
intraventricular subdural any                                   

In [44]:
df_new_lb['labels'].value_counts().size

31

In [5]:
labels = list(df_new_lb['labels'].value_counts().index)[::-1]
label_dict = {k: i for i, k in enumerate(labels)}
label_dict

{'epidural intraventricular subarachnoid any': 0,
 'epidural intraventricular subdural any': 1,
 'epidural intraparenchymal intraventricular subdural any': 2,
 'epidural intraparenchymal intraventricular subarachnoid subdural any': 3,
 'epidural intraventricular subarachnoid subdural any': 4,
 'epidural intraparenchymal intraventricular subarachnoid any': 5,
 'epidural intraventricular any': 6,
 'epidural intraparenchymal subarachnoid subdural any': 7,
 'epidural intraparenchymal intraventricular any': 8,
 'epidural intraparenchymal subarachnoid any': 9,
 'epidural intraparenchymal subdural any': 10,
 'epidural subarachnoid subdural any': 11,
 'epidural subarachnoid any': 12,
 'epidural intraparenchymal any': 13,
 'epidural subdural any': 14,
 'intraparenchymal intraventricular subdural any': 15,
 'intraparenchymal intraventricular subarachnoid subdural any': 16,
 'intraventricular subarachnoid subdural any': 17,
 'intraventricular subdural any': 18,
 'epidural any': 19,
 'intraparench

In [6]:
lbl = df_new_lb['labels'].value_counts().index[-1]; lbl

'epidural intraventricular subarachnoid any'

In [7]:
lbl_id = df_new_lb[df_new_lb['labels'] == lbl]['PatientID'].iloc[0]; lbl_id

'ID_c9942a18'

In [8]:
df_new_lb[df_new_lb['PatientID'] ==lbl_id]

,PatientID,labels
7673,ID_c9942a18,NaN
28748,ID_c9942a18,epidural subarachnoid any
67901,ID_c9942a18,NaN
95911,ID_c9942a18,epidural intraventricular subarachnoid any
106783,ID_c9942a18,NaN
112462,ID_c9942a18,NaN
148412,ID_c9942a18,epidural intraventricular subarachnoid any
156365,ID_c9942a18,epidural intraventricular subarachnoid any
177443,ID_c9942a18,NaN
220859,ID_c9942a18,intraventricular any


In [9]:
df_group = pd.DataFrame(df_new_lb.dropna().groupby('PatientID')['labels'].apply(lambda x: list(x.unique())))

In [10]:
df_group['labels'][df_group['labels'].apply(lambda x: lbl in x)]

PatientID
ID_8fd2b487    [epidural subarachnoid any, epidural any, epid...
ID_b4f7750e    [subarachnoid any, intraparenchymal any, intra...
ID_c9942a18    [epidural subarachnoid any, epidural intravent...
Name: labels, dtype: object

In [11]:
df_group['labels'][df_group['labels'].apply(lambda x: lbl in x)].index.sort_values() 

Index(['ID_8fd2b487', 'ID_b4f7750e', 'ID_c9942a18'], dtype='object', name='PatientID')

In [12]:
df_new_lb[df_new_lb['labels'] == lbl]['PatientID'].sort_values().unique()

array(['ID_8fd2b487', 'ID_b4f7750e', 'ID_c9942a18'], dtype=object)

In [16]:
df_temp = df_group.copy()

In [17]:
df_temp['labels'][df_temp['labels'].apply(lambda x: lbl in x)]

PatientID
ID_8fd2b487    [epidural subarachnoid any, epidural any, epid...
ID_b4f7750e    [subarachnoid any, intraparenchymal any, intra...
ID_c9942a18    [epidural subarachnoid any, epidural intravent...
Name: labels, dtype: object

In [18]:
ar_ids = df_temp['labels'][df_temp['labels'].apply(lambda x: lbl in x)].index.to_numpy()
split_num = max(int(ar_ids.size *0.1), 1)
val_usr_ids =   list(np.random.choice(ar_ids, split_num, replace=False))

In [19]:
ar_ids

array(['ID_8fd2b487', 'ID_b4f7750e', 'ID_c9942a18'], dtype=object)

In [20]:
split_num

1

In [21]:
val_usr_ids

['ID_b4f7750e']

In [22]:
df_temp.drop(ar_ids)

,labels,class_id
PatientID,,
ID_0027f0b7,[subarachnoid any],0
ID_00307f7a,"[subarachnoid any, epidural any, epidural suba...",0
ID_0032e9ab,"[intraparenchymal intraventricular any, intrav...",0
ID_004ebc9b,[subdural any],0
ID_00526c11,"[subdural any, subarachnoid any, intraparenchy...",0
...,...,...
ID_ffd91f66,"[intraparenchymal intraventricular any, intrap...",0
ID_ffe27487,"[subdural any, epidural any, epidural subdural...",0
ID_ffe3608c,[subdural any],0


In [23]:
def get_val_labels(df, labels_list):
    val_indx = []
    df_temp = df.copy()
    for lbl in labels_list:
        ar_ids = df_temp['labels'][df_temp['labels'].apply(lambda x: lbl in x)].index.to_numpy()
        split_num = max(int(ar_ids.size *0.15), 1)
        val_usr_ids =   list(np.random.choice(ar_ids, split_num, replace=False))
        val_indx.append(val_usr_ids)
        df_temp.drop(ar_ids, inplace=True)
        
    return val_indx

In [24]:
vi = np.concatenate(get_val_labels(df_temp, labels))

In [25]:
vi.size

1038

In [26]:
np.unique(vi).size

1038

In [48]:
df_new_lb.set_index('PatientID', inplace=True)

In [28]:
df_new_lb.loc[vi]['labels'].value_counts()

subdural any                                                            4334
subarachnoid any                                                        2234
intraparenchymal any                                                    2002
intraventricular any                                                    1434
intraparenchymal intraventricular any                                    964
subarachnoid subdural any                                                680
intraventricular subarachnoid any                                        544
intraparenchymal subarachnoid any                                        494
intraparenchymal subdural any                                            394
intraparenchymal subarachnoid subdural any                               315
intraparenchymal intraventricular subarachnoid any                       279
epidural any                                                             224
intraventricular subdural any                                            164

In [29]:
df_group_nas = pd.DataFrame(df_new_lb.groupby('PatientID')['labels'].apply(lambda x: list(x.unique())))

In [31]:
list_of_any = df_group_nas['labels'][df_group_nas['labels'].apply(lambda x: len(x)==1 and np.nan in x)][:1500].index

In [33]:
final_val_indx = np.concatenate([vi, list_of_any])

In [35]:
len(final_val_indx)

2538

In [37]:
len(set(final_val_indx))

2538

In [39]:
df_new_lb.loc[final_val_indx]['labels'].value_counts().size == 31

True

In [49]:
df_new_lb.loc[final_val_indx]

,labels
PatientID,
ID_8fd2b487,NaN
ID_8fd2b487,NaN
ID_8fd2b487,NaN
ID_8fd2b487,NaN
ID_8fd2b487,epidural subarachnoid any
...,...
ID_24e1e06f,NaN
ID_24e1e06f,NaN
ID_24e1e06f,NaN


In [61]:
df_new.reset_index(inplace=True)
df_new.set_index('PatientID', inplace=True)

In [62]:
df_new.loc[final_val_indx]

,index,fn,labels
PatientID,,,
ID_8fd2b487,10842,ID_e2f060c42.png,NaN
ID_8fd2b487,26058,ID_c9db7caa5.png,NaN
ID_8fd2b487,54555,ID_d983bd1fc.png,NaN
ID_8fd2b487,71847,ID_d869d8af7.png,NaN
ID_8fd2b487,82176,ID_11a5dd378.png,epidural subarachnoid any
...,...,...,...
ID_24e1e06f,557455,ID_e6313a792.png,NaN
ID_24e1e06f,572204,ID_6fef16f36.png,NaN
ID_24e1e06f,580513,ID_40b6806a5.png,NaN


In [64]:
df_new.loc[final_val_indx]

,index,fn,labels
PatientID,,,
ID_8fd2b487,10842,ID_e2f060c42.png,NaN
ID_8fd2b487,26058,ID_c9db7caa5.png,NaN
ID_8fd2b487,54555,ID_d983bd1fc.png,NaN
ID_8fd2b487,71847,ID_d869d8af7.png,NaN
ID_8fd2b487,82176,ID_11a5dd378.png,epidural subarachnoid any
...,...,...,...
ID_24e1e06f,557455,ID_e6313a792.png,NaN
ID_24e1e06f,572204,ID_6fef16f36.png,NaN
ID_24e1e06f,580513,ID_40b6806a5.png,NaN


In [69]:
val_idx = df_new.loc[final_val_indx]['index'].values

In [70]:
val_idx

array([ 10842,  26058,  54555, ..., 580513, 591623, 665927])

In [82]:
df_new.drop(final_val_indx, inplace=True)

In [85]:
trn_idx = df_new['index'].values

In [92]:
joblob_idx = (val_idx, trn_idx)

In [90]:
val_idx.size + trn_idx.size == df.shape[0]

True

In [93]:
df

,fn,labels
0,ID_63eb1e259.png,NaN
1,ID_2669954a7.png,NaN
2,ID_52c9913b1.png,NaN
3,ID_4e6ff6126.png,NaN
4,ID_7858edd88.png,NaN
...,...,...
674252,ID_f737f4cc1.png,NaN
674253,ID_4c92d70b5.png,NaN
674254,ID_a9797cb3a.png,NaN
674255,ID_9375f67bd.png,NaN


In [95]:
joblob_idx[0]

array([ 10842,  26058,  54555, ..., 580513, 591623, 665927])

In [98]:
df.iloc[joblob_idx[0]]['labels'].value_counts()

subdural any                                                            4334
subarachnoid any                                                        2234
intraparenchymal any                                                    2002
intraventricular any                                                    1434
intraparenchymal intraventricular any                                    964
subarachnoid subdural any                                                680
intraventricular subarachnoid any                                        544
intraparenchymal subarachnoid any                                        494
intraparenchymal subdural any                                            394
intraparenchymal subarachnoid subdural any                               315
intraparenchymal intraventricular subarachnoid any                       279
epidural any                                                             224
intraventricular subdural any                                            164

In [99]:
df.iloc[joblob_idx[1]]['labels'].value_counts()

subdural any                                                            24575
subarachnoid any                                                        12483
intraparenchymal any                                                    11930
intraventricular any                                                     7480
intraparenchymal intraventricular any                                    5366
subarachnoid subdural any                                                3974
intraparenchymal subarachnoid any                                        3117
intraventricular subarachnoid any                                        2682
intraparenchymal subdural any                                            2558
intraparenchymal subarachnoid subdural any                               1702
intraparenchymal intraventricular subarachnoid any                       1482
epidural any                                                             1273
intraventricular subdural any                                   

In [100]:
dump(joblob_idx, 'VAL_by_Patience/val_idx_pi.joblib')

['VAL_by_Patience/val_idx_pi.joblib']